<a href="https://colab.research.google.com/github/AhmadKElsayed/YouTubeChecker/blob/Agent/YoutubeChecker_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
[YouTube URL] → [Extract Transcript]
                      ↓
                  [NER Pipeline]
                      ↓
               [Extract Facts / Claims]
                      ↓
             [Fact-check with Gemini]
                      ↓
         [Optionally: Google Search / SerpAPI]
                      ↓
       [Return verdict + explanation + URL]
'''


'\n[YouTube URL] → [Extract Transcript]\n                      ↓\n                  [NER Pipeline]\n                      ↓\n               [Extract Facts / Claims]\n                      ↓\n             [Fact-check with Gemini]\n                      ↓\n         [Optionally: Google Search / SerpAPI]\n                      ↓\n       [Return verdict + explanation + URL]\n'

In [ ]:
!pip install google-generativeai
!pip install youtube-transcript-api
!pip install langchain
!pip install langchain-google-genai
# !pip install youtube-transcript-api
!pip install transformers
# !pip install serpapi
!pip install dateparser
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install -U langchain-community
!pip install duckduckgo-search
!pip install google-search-results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=7e32d6bce0ad9d3201e1c1b8a6fa48c1534a

In [ ]:
import os
import dateparser
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
import spacy
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""
video_url = "https://www.youtube.com/watch?v=JIbIYCM48to&t=99s"

In [ ]:
def get_youtube_transcript(video_url):

    video_id = video_url.split("v=")[-1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([item['text'] for item in transcript])

    return text
text = get_youtube_transcript(video_url)
print(text)

Amazon web services launched in 2006 with a total of three products storage buckets compute instances and a messaging queue today it offers a mind-numbing 200 and something services and what's most confusing is that many of them appear to do almost the exact same thing it's kind of like shopping at a big grocery store where you have different aisles of product categories filled with things to buy that meet the needs of virtually every developer on the planet in today's video we'll walk down these aisles to gain an understanding of over 50 different AWS products so first let's start with a few that are above my pay grade that you may not know exist if you're building robots you can use robomaker to simulate and test your robots at scale then once your robots are in people's homes you can use iot core to collect data from them update their software and manage them remotely if you happen to have a satellite orbiting Earth you can tap into Amazon's Global Network of antennas to connect dat

In [ ]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Load spaCy and Hugging Face NER model
nlp = spacy.load("en_core_web_sm")

from transformers import pipeline
import spacy
import dateparser

# Load models
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")
nlp = spacy.load("en_core_web_sm")

def extract_factual_claims(text):
    # Process the transcript into sentences
    doc = nlp(text)
    extracted_claims = []

    for sent in doc.sents:
        sentence_text = sent.text.strip()
        if len(sentence_text) < 5:
            continue

        # Run NER on each sentence individually
        entities = ner_pipeline(sentence_text)

        # Only keep sentences with detected entities
        if entities:
            claim_info = {
                "sentence": sentence_text,
                "entities": []
            }
            for ent in entities:
                entity_data = {
                    "text": ent['word'],
                    "type": ent['entity_group'],
                    "score": ent['score']
                }
                if ent['entity_group'] == "DATE":
                    parsed_date = dateparser.parse(ent['word'])
                    entity_data["parsed_date"] = str(parsed_date) if parsed_date else None
                claim_info["entities"].append(entity_data)
            extracted_claims.append(claim_info)

    return extracted_claims


# Example
claims = extract_factual_claims(text)
for c in claims[:5]:
    print(c)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'sentence': "Amazon web services launched in 2006 with a total of three products storage buckets compute instances and a messaging queue today it offers a mind-numbing 200 and something services and what's most confusing is that many of them appear to do almost the exact same thing it's kind of like shopping at a big grocery store where you have different aisles of product categories filled with things to buy that meet the needs of virtually every developer on the planet in today's video we'll walk down these aisles to gain an understanding of over 50 different AWS products so first let's start with a few that are above my pay grade that you may not know exist if you're building robots you can use robomaker to simulate and test your robots at scale then once your robots are in people's homes you can use iot core to collect data from them update their software and manage them remotely if you happen to have a satellite orbiting Earth you can tap into Amazon's Global Network of antennas 

# AGENT:

##LLM:

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash-lite-preview", temperature=0.2)

## Tools:

In [ ]:
Youtube_Transcriptor = Tool(name="YouTube Transcript",func=get_youtube_transcript,description="Fetch transcript from a YouTube video given its video URL")

In [ ]:
search = DuckDuckGoSearchRun()
DDSearch = Tool(name="Search", func=search.run, description="Useful for looking up facts or credible URLs"  )

In [ ]:
fact_claim_extractor = Tool(
    name="FactClaimExtractor",
    func=lambda text: str(extract_factual_claims(text)),  # return as stringified JSON for LLM
    description="Extracts factual claims and named entities from a given text transcript. Expects a plain text transcript."
)

In [ ]:
tools = [
    fact_claim_extractor,
    DDSearch
]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-13-9bbf088a4509>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
def fact_check_claim(transcript):

    prompt = f"""
    Fact-check this transcript: "{transcript}"
    First, extract any factual claims or named entities using FactClaimExtractor.
    Then, fact-check those claims one by one.
    - Respond with 'Correct', 'Incorrect', or 'Unverifiable'.
    - Explain briefly why.
    - Get me three credible URL on your own.
    - If you couldn't find them on your own, call the Search tool to look up credible sources and then respond.
    """
    result = agent.run(prompt)
    return result

result = fact_check_claim("Paris is in Italy. Napoleon was born there")
print(result)



> Entering new AgentExecutor chain...


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
# Example Usage
video_url = "https://www.youtube.com/watch?v=JIbIYCM48to&t=99s"
transcript = get_youtube_transcript(video_url)
facts = extract_factual_claims(transcript)

for fact in facts[:5]:
    print(f"📝 Fact: {fact}")
    result = fact_check_claim(fact)
    print(result)
    print("-" * 50)
    time.sleep(5)  # wait 5 seconds between calls

In [ ]:
from google.generativeai import GenerativeModel
import google.generativeai # Import the main library

# List available models
# models = GenerativeModel.list() # Incorrect way to list models
models = google.generativeai.list_models() # Correct way to list models
for m in models:
    print(m.name, m.description)

models/chat-bison-001 A legacy text-only model optimized for chat conversations
models/text-bison-001 A legacy model that understands text and generates text as an output
models/embedding-gecko-001 Obtain a distributed representation of a text.
models/gemini-1.0-pro-vision-latest The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
models/gemini-pro-vision The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
models/gemini-1.5-pro-latest Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens.
models/gemini-1.5-pro-001 Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in M